In [1]:
import sys
from PyQt5.QtWidgets import QApplication, QWidget, QVBoxLayout, QHBoxLayout, QPushButton, QTextEdit, QLabel, QFileDialog
from PyQt5.QtGui import QPixmap
from PIL import Image
import numpy as np

class LSBSteganographyGUI(QWidget):
    def __init__(self):
        super().__init__()
        self.initUI()

    def initUI(self):
        self.setWindowTitle('LSB Steganography')
        self.setGeometry(100, 100, 800, 600)

        layout = QVBoxLayout()

        # Image selection
        self.image_label = QLabel()
        self.image_label.setFixedSize(300, 300)
        self.select_image_button = QPushButton('Select Image')
        self.select_image_button.clicked.connect(self.select_image)
        layout.addWidget(self.image_label)
        layout.addWidget(self.select_image_button)

        # Message input
        self.message_input = QTextEdit()
        self.message_input.setPlaceholderText("Enter message to hide")
        layout.addWidget(QLabel("Message:"))
        layout.addWidget(self.message_input)

        # Buttons
        button_layout = QHBoxLayout()
        self.hide_button = QPushButton('Hide Message')
        self.extract_button = QPushButton('Extract Message')
        self.hide_button.clicked.connect(self.hide_message)
        self.extract_button.clicked.connect(self.extract_message)
        button_layout.addWidget(self.hide_button)
        button_layout.addWidget(self.extract_button)
        layout.addLayout(button_layout)

        # Output
        self.output_text = QTextEdit()
        self.output_text.setReadOnly(True)
        layout.addWidget(QLabel("Output:"))
        layout.addWidget(self.output_text)

        self.setLayout(layout)

    def select_image(self):
        file_name, _ = QFileDialog.getOpenFileName(self, "Select Image", "", "Image Files (*.png *.jpg *.bmp)")
        if file_name:
            pixmap = QPixmap(file_name)
            self.image_label.setPixmap(pixmap.scaled(self.image_label.size(), aspectRatioMode=True))
            self.image_path = file_name

    def hide_message(self):
        if not hasattr(self, 'image_path'):
            self.output_text.setText("Please select an image first.")
            return

        message = self.message_input.toPlainText()
        if not message:
            self.output_text.setText("Please enter a message to hide.")
            return

        img = Image.open(self.image_path)
        img_array = np.array(list(img.getdata()))

        if img.mode == 'RGB':
            n = 3
        elif img.mode == 'RGBA':
            n = 4

        total_pixels = img_array.size // n

        message += "###"
        b_message = ''.join([format(ord(i), "08b") for i in message])
        req_pixels = len(b_message)

        if req_pixels > total_pixels:
            self.output_text.setText("Message too large for the image.")
            return

        index = 0
        for p in range(total_pixels):
            for q in range(0, 3):
                if index < req_pixels:
                    img_array[p][q] = int(bin(img_array[p][q])[2:9] + b_message[index], 2)
                    index += 1

        img_array = img_array.reshape(img.size[1], img.size[0], n)
        result = Image.fromarray(img_array.astype('uint8'), img.mode)
        result.save("stego_image.png")
        self.output_text.setText("Message hidden successfully. Saved as 'stego_image.png'")

    def extract_message(self):
        if not hasattr(self, 'image_path'):
            self.output_text.setText("Please select an image first.")
            return

        img = Image.open(self.image_path)
        img_array = np.array(list(img.getdata()))

        if img.mode == 'RGB':
            n = 3
        elif img.mode == 'RGBA':
            n = 4

        total_pixels = img_array.size // n

        hidden_bits = ""
        for p in range(total_pixels):
            for q in range(0, 3):
                hidden_bits += (bin(img_array[p][q])[2:][-1])

        hidden_bits = [hidden_bits[i:i+8] for i in range(0, len(hidden_bits), 8)]
        message = ""
        for i in range(len(hidden_bits)):
            if message[-3:] == "###":
                break
            else:
                message += chr(int(hidden_bits[i], 2))

        if "###" in message:
            self.output_text.setText(message[:-3])
        else:
            self.output_text.setText("No hidden message found.")

if __name__ == '__main__':
    app = QApplication(sys.argv)
    ex = LSBSteganographyGUI()
    ex.show()
    sys.exit(app.exec_())

SystemExit: 0

C:\Users\LENOVO\anaconda3\Lib\site-packages\IPython\core\interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
